In [1]:
!nvidia-smi

Wed Jun 15 22:01:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# import tensorflow as tf
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
# with tpu_strategy.scope():
#     model = tf.keras.Sequential( … ) # define your model normally
#     model.compile( … )
    
# model.fit(training_dataset, epochs=EPOCHS, steps_per_epoch=…)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/abstract'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        print(dirname)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/abstract/sample_submission.csv
/kaggle/input/abstract
/kaggle/input/abstract/train.csv
/kaggle/input/abstract
/kaggle/input/abstract/test.csv
/kaggle/input/abstract


In [2]:
from sklearn.model_selection import train_test_split
trainData = pd.read_csv('/kaggle/input/abstract/train.csv', sep=',', header="infer")
testData = pd.read_csv('/kaggle/input/abstract/test.csv', sep = ',', header = "infer")
testLabels = pd.read_csv('/kaggle/input/abstract/sample_submission.csv', sep = ',', header = "infer")

In [3]:
train_data, remaining = train_test_split(trainData, train_size=0.85, random_state=34)
test_data, val_data = train_test_split(remaining, train_size=0.7, random_state=34)
train_data.shape, val_data.shape, test_data.shape

((18874, 9), (2098, 9))

# Dataset Exploration

In [ ]:
trainData.head()

In [ ]:
trainData.describe()

In [ ]:
len(trainData[trainData['Computer Science'] == 1])

In [ ]:
len(trainData[trainData['Physics'] == 1])

In [ ]:
len(trainData[trainData['Mathematics'] == 1])

In [ ]:
len(trainData[trainData['Statistics'] == 1])

In [ ]:
len(trainData[trainData['Quantitative Biology'] == 1])

In [ ]:
len(trainData[trainData['Quantitative Finance'] == 1])

In [ ]:
list(trainData.groupby(['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']).aggregate('Quantitative Finance'))

In [ ]:
import numpy as np
np.where(trainData.iloc[:,3:].values == [0,0,1,1,0,1])

In [ ]:
trainData.columns.values

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

# -> Plot of class distributions

In [ ]:
categories = list(trainData.columns.values[3:])
sns.set(font_scale = 1)
plt.figure(figsize=(15,8))
ax= sns.barplot(x = categories, y = trainData.iloc[:,3:].sum().values)
plt.title("Abstract of each category", fontsize=24)
plt.ylabel('Number of abstracts', fontsize=18)
plt.xlabel('Abstract Type ', fontsize=18)
#adding the text labels
rects = ax.patches
labels = trainData.iloc[:,3:].sum().values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom', fontsize=18)
    
plt.show()


In [ ]:
sns.set(font_scale = 1)
plt.figure(figsize=(15,8))
multiLabel_counts = trainData.iloc[:,3:].sum(axis=1).value_counts()

ax = sns.barplot(x = multiLabel_counts.index,y = multiLabel_counts)

plt.title("Abstracts with multiple labels ")
plt.ylabel('Number of Abstracts', fontsize=18)
plt.xlabel('Number of labels', fontsize=18)
#adding the text labels
rects = ax.patches
labels = multiLabel_counts.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')
plt.show()

In [ ]:
'./model/+{model_name}'

# Transformer Model Implementation From Hugging Face

In [1]:
!pip install focal_loss
from focal_loss import BinaryFocalLoss
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification, DistilBertTokenizerFast
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pickle

print("TF Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


class DistilBertTrain:
    def __init__(self):
        self.trainData = pd.read_csv('/kaggle/input/abstract/train.csv', sep=',', header="infer")
        self.trainData_cp = self.trainData.copy()
        
#         self.testData = pd.read_csv('/kaggle/input/abstract/test.csv', sep = ',', header = "infer")
#         self.testLabels = pd.read_csv('/kaggle/input/abstract/sample_submission.csv', sep = ',', header = "infer")
        self.MODEL_NAME = 'distilbert-base-uncased'
        self.X = 'ABSTRACT'
        self.Y = [3,4,5,6,7,8]
        self.num_classes = len(self.Y)
        self.BATCH_SIZE = 16
        self.MAX_LENGTH = 512
        self.N_EPOCHS = 5
        self.lr=1e-5
        self.tokenizer = DistilBertTokenizerFast.from_pretrained(self.MODEL_NAME)
        self.model = TFDistilBertForSequenceClassification.from_pretrained(self.MODEL_NAME, num_labels = self.num_classes)
        
    def preprocess(self):
        self.trainData_cp[self.X] = self.trainData[self.X].apply(lambda x: re.sub("\n"," ",x))
        train_data, val_data = train_test_split(self.trainData_cp, train_size=0.85, random_state=34)
        val_data, test_data = train_test_split(val_data, train_size=0.7, random_state=34)
        return train_data, val_data, test_data
        
    def distilBertTokenization(self, train_data, val_data):
        train_encodings = self.tokenizer(train_data.ABSTRACT.to_list(), truncation=True, padding=True)
        val_encodings = self.tokenizer(val_data.ABSTRACT.to_list(), truncation=True, padding=True)
        return train_encodings, val_encodings
    
    def distilBertPipelineGeneration(self, train_encodings, val_encodings, train_data, val_data):
        train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),
                                    list(train_data.iloc[:,3:9].values)))
        val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings),
                                    list(val_data.iloc[:,3:9].values)))
        # train
        tr_pipe = (train_dataset.shuffle(len(train_data.ABSTRACT))
                  .batch(self.BATCH_SIZE, drop_remainder=True)
                    .prefetch(tf.data.experimental.AUTOTUNE)
                      )

  # valid
        val_pipe = (val_dataset.batch(self.BATCH_SIZE, drop_remainder=True)
                  .prefetch(tf.data.experimental.AUTOTUNE)
                    )
    
        return tr_pipe, val_pipe
    
    def fit(self, tr_data, vl_data):
        def scheduler(epoch, lr):
            if epoch < 2:
                return lr
            else:
                return lr * tf.math.exp(-0.1)
        
        optimizer = tf.keras.optimizers.Adam(learning_rate= self.lr)

        earlystp = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    min_delta=0,
                    patience=3,
                    verbose=1,
                    mode='auto',
                    baseline=None,
                    restore_best_weights=True)



        lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler)

        loss = BinaryFocalLoss(gamma=2)

        self.model.compile(optimizer, loss, metrics=['accuracy'])
        self.model.fit(tr_data, epochs=self.N_EPOCHS, batch_size=self.BATCH_SIZE, validation_data = vl_data, callbacks =[lr_schedule, earlystp], verbose=1)
        return self.model
    
    def save(self):
        model_name = 'distilbert_base_uncased_model'
        self.model.save_pretrained('./model/+{model_name}+')
        with open('./model/info.pkl', 'wb') as f:
            pickle.dump(('distilbert_base_uncased_model', self.MAX_LENGTH), f)
            
#     def load(self):
#         new_model = TFDistilBertForSequenceClassification.from_pretrained('./model/distilbert_base_uncased_model')
#         self.model_name, self.MAX_LENGTH = pickle.load(open('./model/info.pkl', 'rb'))
            
    



  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 60.4 MB/s eta 0:00:0000:0100:01
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=77037 sha256=1459db870ab610294aa21e4eef24a01efb1ffaa7df3214039984ba7e35cef87a
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.14.1
    Uninstalling wrapt-1.14.1:
      Successfully uninstalled wrapt-1.14.1
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.2.0
    Uninstalling typing_extensions-4.2.0:
      Successfully uninstalled typing_extensions-4.2.0
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.1

2022-06-16 22:10:23.250951: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 22:10:23.252028: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 22:10:23.252742: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# Class instance creation and invokation

In [2]:
model = DistilBertTrain()

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

2022-06-16 22:11:14.440634: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-16 22:11:14.441123: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 22:11:14.442347: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 22:11:14.443326: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

# Data Preprocessing

In [3]:
train_data, val_data, test_data = model.preprocess()

# DistilBERT Tokenization

In [4]:
train_encodings, val_encodings = model.distilBertTokenization(train_data, val_data)

# Tenforflow IO Pipeline creation using tf.data 

In [5]:
tr_pipe, val_pipe = model.distilBertPipelineGeneration(train_encodings, val_encodings, train_data, val_data)

# Model Training

In [6]:
distilBert=model.fit(tr_pipe, val_pipe)

Epoch 1/5


2022-06-16 22:12:45.909910: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


1114/1114 [==============================] - 613s 541ms/step - loss: 0.0868 - accuracy: 0.6978 - val_loss: 0.0496 - val_accuracy: 0.7614
Epoch 2/5
1114/1114 [==============================] - 602s 541ms/step - loss: 0.0500 - accuracy: 0.7637 - val_loss: 0.0478 - val_accuracy: 0.7632
Epoch 3/5
1114/1114 [==============================] - 603s 541ms/step - loss: 0.0441 - accuracy: 0.7751 - val_loss: 0.0539 - val_accuracy: 0.7450
Epoch 4/5
1114/1114 [==============================] - 602s 541ms/step - loss: 0.0406 - accuracy: 0.7904 - val_loss: 0.0479 - val_accuracy: 0.7340
Epoch 5/5
1114/1114 [==============================] - 603s 541ms/step - loss: 0.0357 - accuracy: 0.7962 - val_loss: 0.0643 - val_accuracy: 0.7774
Restoring model weights from the end of the best epoch.
Epoch 00005: early stopping


In [8]:
# model_name = 'distilbert_base_uncased_model'
distilBert.save_pretrained('./model/distilbert_base_uncased_model')
with open('./model/info.pkl', 'wb') as f:
    pickle.dump(('distilbert_base_uncased_model', 512), f)

# Model Prediction

In [ ]:
# class DistilBertPredict:
#     def __init__(self, model, test_data):
#         self.
    
    
#     def predict(self, test_data):

In [22]:
DistilBertConfig()

DistilBertConfig {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "transformers_version": "4.18.0",
  "vocab_size": 30522
}

# Dynamic sentence splitting and check for the confidence on each consecutive addition to last sentence

In [7]:
import spacy
# ! python -m spacy download en_core_web_lg
# spacy_lg = spacy.load('en_core_web_lg')
! python -m spacy download en_core_web_sm
spacy_sm = spacy.load('en_core_web_sm')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 31.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:

def dynamicSentenceSelection(text, model):
    sent_list = list(spacy_sm(train_data.ABSTRACT[0]).sents)
    num_sents = len(sent_list)
    prob_list = np.zeros([num_sents])
    
    for i in np.arange(num_sents):
        np.append(prob_list, model.predict(sent_list[:i]))
        
    np.mean(prob_list)
    return prob_list
        
dynamicSentenceSelection(train_data.ABSTRACT[0])

  Predictive models allow subject-specific inference when analyzing disease
related alterations in neuroimaging data.
Given a subject's data, inference can
be made at two levels: global, i.e. identifiying condition presence for the
subject, and local, i.e. detecting condition effect on each individual
measurement extracted from the subject's data.
While global inference is widely
used, local inference, which can be used to form subject-specific effect maps,
is rarely used because existing models often yield noisy detections composed of
dispersed isolated islands.
In this article, we propose a reconstruction
method, named RSM, to improve subject-specific detections of predictive
modeling approaches and in particular, binary classifiers.
RSM specifically
aims to reduce noise due to sampling error associated with using a finite
sample of examples to train classifiers.
The proposed method is a wrapper-type
algorithm that can be used with different binary classifiers in a diagnostic
manner,

In [ ]:
Hyperparameter Tuning

1. Learning rate
2. Early stopping
3. Gradient clipping
4. number of epochs to train
5. dropout
6. regularization